In [1]:
import glob
import json
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import nibabel as nib
from nilearn.glm.first_level import FirstLevelModel
from nilearn.glm.first_level import make_first_level_design_matrix
from nilearn.plotting import plot_design_matrix
import os
import pandas as pd


/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/glm/__init__.py:55: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  warn('The nilearn.glm module is experimental. '
/Users/zeynepenkavi/.pyenv/versions/3.8.3/envs/py-dvl-fmri/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
import os
from pyhere import here
import sys
sys.path.append(str(here('01_level1')))
from level1_utils import run_level1

In [3]:
subnum = '01'
mnum = 'model8'
runnum = '1'

base_path = '/Users/zeynepenkavi/Downloads/GTavares_2017_arbitration/'
data_path = os.path.join(base_path, 'bids_nifti_wface/')
behavior_path = os.path.join(base_path, 'behavioral_data/all_trials.csv')
out_path = os.path.join(data_path, 'derivatives/nilearn/glm/level1/%s_reg-rt%s'%(mnum, "0"))

In [8]:
fn = os.path.join(data_path, 'sub-%s/func/sub-%s_task-bundles_run-%s_events.tsv' %(subnum, subnum, runnum))
events = pd.read_csv(fn, sep='\t')

# Read in behavioral data with modeled value and RPE estimates
behavior = pd.read_csv(behavior_path)

# Extract the correct subnum and runnum from behavioral data
run_behavior = behavior.query('subnum == %d & session == %d'%(int(subnum), int(runnum)))

# Demean columns that might be used for parametric regressors
demean_cols = ['probFractalDraw', 'reward', 'leftBundleValAdv','leftFractalRpe', 'rightFractalRpe', 'rpeLeftRightSum','valChosen', 'valUnchosen', 'valChosenLottery', 'valUnchosenLottery', 'valChosenFractal', 'valUnchosenFractal', 'valBundleSum', 'valChosenMinusUnchosen']
demean_df = run_behavior[demean_cols]
demean_df = demean_df - demean_df.mean()

In [10]:
cond_rpeRelativeLeftFractal_par = events.query('trial_type == "reward"')[['onset', 'duration']].reset_index(drop=True)
cond_rpeRelativeLeftFractal_par['trial_type'] = 'rpeRelativeLeftFractal_par'
cond_rpeRelativeLeftFractal_par['modulation'] = run_behavior['leftFractalRpe'] - run_behavior['rightFractalRpe']
cond_rpeRelativeLeftFractal_par['modulation'] = cond_rpeRelativeLeftFractal_par['modulation'] - cond_rpeRelativeLeftFractal_par['modulation'].mean()
cond_rpeRelativeLeftFractal_par

,onset,duration,trial_type,modulation
0,9.685461,3.018033,rpeRelativeLeftFractal_par,0.037965
1,24.075609,3.018088,rpeRelativeLeftFractal_par,0.037965
2,38.299017,3.018091,rpeRelativeLeftFractal_par,0.037965
3,52.589129,3.018032,rpeRelativeLeftFractal_par,1.037965
4,68.930201,3.018094,rpeRelativeLeftFractal_par,-0.063589
5,85.471396,3.018002,rpeRelativeLeftFractal_par,-0.053276
6,98.744252,3.018180,rpeRelativeLeftFractal_par,-1.044010
7,114.701898,3.018098,rpeRelativeLeftFractal_par,-0.934132
8,130.425988,3.034811,rpeRelativeLeftFractal_par,0.164588
9,147.050568,3.018099,rpeRelativeLeftFractal_par,-0.848271


In [ ]:
from level1_utils import make_level1_design_matrix, get_events, make_contrasts

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model8', data_path, behavior_path, regress_rt = 0)

In [ ]:
contrasts = make_contrasts(design_matrix, 'model8')

In [ ]:
from utils import get_model_regs_with_contrasts, get_model_regs

In [ ]:
get_model_regs_with_contrasts('model8')


In [ ]:
run_level1(subnum, mnum, data_path, behavior_path, out_path, regress_rt=0, save_contrast = True)

In [ ]:
to_filter = ['trans', 'rot', 'drift', 'framewise', 'scrub', 'constant', 'dvars']
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1', data_path, behavior_path, regress_rt = 0)
# plot_design_matrix(design_matrix)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model1a', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix = make_level1_design_matrix(subnum, runnum, 'model2', data_path, behavior_path, regress_rt = 0)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
design_matrix[beh_regs].corr()

In [ ]:
design_matrix= make_level1_design_matrix(subnum, runnum, 'model3', data_path, behavior_path, regress_rt = 0, hrf_model=None)
beh_regs = design_matrix.columns
beh_regs = [x for x in beh_regs if all(y not in x for y in to_filter)]
plot_design_matrix(design_matrix)